In [1]:
import numpy as np
import pyrr as pr
from dmbtools.geometry import utils
from dmbtools.geodetic import convert as cvt

In [2]:
u = np.array([1.0, 1.0, 0.0]) # point on line
print(u)
print()
v = np.array([1.0, 0.0, 0.0]) # direction of line
print(v)
print()
w = np.array([0.0, 1.0, 0.0])
print(w)
print()
n = np.cross(v, w)
print(n)
print()
y = np.array([[0.0, 0.0, 0.0], [2.0, 2.0, 0.0], [3.0, 3.0, 0.0], [4.0, 4.0, 0.0], [5.0, 5.0, 0.0]]) # points to project onto line
print(y)

[1. 1. 0.]

[1. 0. 0.]

[0. 1. 0.]

[0. 0. 1.]

[[0. 0. 0.]
 [2. 2. 0.]
 [3. 3. 0.]
 [4. 4. 0.]
 [5. 5. 0.]]


In [3]:
def _pee_cross2(p):
        """
        http://people.eecs.berkeley.edu/~wkahan/MathH110/Cross.pdf
        p: a column vector.
        """
        return np.array([[  0.0, -p[2],  p[1]], 
                         [ p[2],   0.0, -p[0]], 
                         [-p[1],  p[0],  0.0]])

In [4]:
utils.rtv(v, w, y)

array([[ 0.,  0.,  0.],
       [-2.,  2.,  0.],
       [-3.,  3.,  0.],
       [-4.,  4.,  0.],
       [-5.,  5.,  0.]])

In [5]:
np.linalg.norm(utils.rtv(v, w, y), axis=1)

array([0.        , 2.82842712, 4.24264069, 5.65685425, 7.07106781])

In [6]:
theta = utils.angle(v, w) * 2.5
np.rad2deg(theta)

225.0

In [7]:
np.linalg.norm(utils.rodrigues(theta, n, y), axis=1)

array([0.        , 2.82842712, 4.24264069, 5.65685425, 7.07106781])

In [8]:
utils.gibbs(theta, n, y)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.44089210e-16, -2.82842712e+00,  0.00000000e+00],
       [-4.44089210e-16, -4.24264069e+00,  0.00000000e+00],
       [-8.88178420e-16, -5.65685425e+00,  0.00000000e+00],
       [-1.33226763e-15, -7.07106781e+00,  0.00000000e+00]])

In [9]:
quat = pr.Quaternion.from_axis_rotation(n, theta, dtype=None)
quat

Quaternion([ 0.        ,  0.        ,  0.92387953, -0.38268343])

In [10]:
pr.Vector3(y[1])

Vector3([2., 2., 0.])

In [11]:
v3 = [pr.Vector3(yy) for yy in y]

In [12]:
type(v3[0])

pyrr.objects.vector3.Vector3

In [13]:
quat * v3[1]

Vector3([-1.11022302e-16, -2.82842712e+00,  0.00000000e+00])

In [14]:
[(quat * v3i) for v3i in v3]

[Vector3([0., 0., 0.]),
 Vector3([-1.11022302e-16, -2.82842712e+00,  0.00000000e+00]),
 Vector3([ 0.        , -4.24264069,  0.        ]),
 Vector3([-2.22044605e-16, -5.65685425e+00,  0.00000000e+00]),
 Vector3([-8.88178420e-16, -7.07106781e+00,  0.00000000e+00])]

In [15]:
I  = np.array([[1.0, 0.0, 0.0],
               [0.0, 1.0, 0.0],
               [0.0, 0.0, 1.0]])

In [16]:
Q = np.tan(theta/2.0) * n
Q

array([-0.        , -0.        , -2.41421356])

In [17]:
Qx = _pee_cross2(Q)
Qx

array([[ 0.        ,  2.41421356, -0.        ],
       [-2.41421356,  0.        ,  0.        ],
       [ 0.        , -0.        ,  0.        ]])

In [18]:
t1 = (2.0 / (1.0 + Q.dot(Q)))
t1

0.2928932188134524

In [19]:
t2 = (Qx + Qx @ Qx)
t2

array([[-5.82842712,  2.41421356,  0.        ],
       [-2.41421356, -5.82842712,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [20]:
RQ = I + t1 * t2
RQ

array([[-0.70710678,  0.70710678,  0.        ],
       [-0.70710678, -0.70710678,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [21]:
(RQ @ y.T).T

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.44089210e-16, -2.82842712e+00,  0.00000000e+00],
       [-4.44089210e-16, -4.24264069e+00,  0.00000000e+00],
       [-8.88178420e-16, -5.65685425e+00,  0.00000000e+00],
       [-1.33226763e-15, -7.07106781e+00,  0.00000000e+00]])